In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
%matplotlib inline

In [219]:
df = pd.read_csv('/Users/veronicadelgado/code/vdelgadobenito/psycovid/raw_data/COVIDiSTRESS_May_30_cleaned_final.csv', encoding='latin-1')
df.head()

/Users/veronicadelgado/.pyenv/versions/3.8.6/envs/psycovid/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (55,56,57,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,answered_all,Duration..in.seconds.,RecordedDate,UserLanguage,Dem_age,Dem_gender,Dem_edu,Dem_edu_mom,Dem_employment,...,Final_open,PSS10_avg,SLON3_avg,neu,ext,ope,agr,con,SPS_avg,Scale_UCLA_TRI_avg
0,1,No,180,2020-05-30 23:47:17,SAR,29,Female,"College degree, bachelor, master",Some College or equivalent,Not employed,...,NaN,2.900000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,No,3100,2020-05-29 23:30:15,UR,20,Male,"College degree, bachelor, master",None,Student,...,NaN,2.200000,2.333333,2.000000,5.000000,5.333333,5.0,5.000000,5.0,NaN
2,3,No,127,2020-05-30 22:40:15,SAR,47,Female,"Some College, short continuing education or eq...",Some College or equivalent,Self-employed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,No,1710,2020-05-29 22:47:17,BG,79,Male,"College degree, bachelor, master",College degree,Not employed,...,NaN,3.600000,4.000000,4.000000,4.666667,4.000000,5.0,5.000000,3.9,NaN
4,5,No,2239,2020-05-29 22:42:30,SAR,61,Female,"Some College, short continuing education or eq...",Up to 12 years of school,Retired,...,Informaciones no confiables,2.714286,1.000000,3.666667,4.666667,5.333333,5.0,4.666667,5.0,NaN


In [220]:
df.shape

(125306, 154)

In [221]:
# Create a copy of the df
# Drop columns that we for sure know we are NOT going to use

unfiltered_data = df.copy()
unfiltered_data.drop(columns=['ID','answered_all','Duration..in.seconds.','RecordedDate','experience_war','born_92','experience_war','experience_war_TXT','war_injury','loss_during_war','time_spent_in_war','time_spent_in_war_TXT','Expl_coping_txt','Expl_Distress_txt','Final_open','AD_gain','AD_loss','AD_check'],inplace=True)

In [222]:
unfiltered_data.shape

(125306, 137)

In [223]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.00244202193031459
Dem_edu 0.006695609148803729
Dem_edu_mom 0.017102133976026685
Dem_employment 0.009073787368521859
Country 0.004349352784383829
Dem_Expat 0.005682090243084928
Dem_state 0.10784798812506984
Dem_maritalstatus 0.008100170781925846
Dem_dependents 0.03072478572454631
Dem_riskgroup 0.004939907107401082
Dem_isolation 0.014229166999185992
Dem_isolation_adults 0.17050260961167063
Dem_isolation_kids 0.1842130464622604
Scale_PSS10_UCLA_1 0.0802036614368027
Scale_PSS10_UCLA_2 0.08045105581536399
Scale_PSS10_UCLA_3 0.08007597401560979
Scale_PSS10_UCLA_4 0.0808979617895392
Scale_PSS10_UCLA_5 0.0818955197676089
Scale_PSS10_UCLA_6 0.08013981772620625
Scale_PSS10_UCLA_7 0.08143265286578456
Scale_PSS10_UCLA_8 0.08177581281024053
Scale_PSS10_UCLA_9 0.07974877499880292
Scale_PSS10_UCLA_10 0.08001213030501332
Scale_SLON_1 0.08077027436834629
Scale_SLON_2 0.08136082869136355
Scale_SLON_3 0.08092190318101287
OECD_people_1 0.11275597337717268
OECD_peo

In [224]:
# Drop columns were %NaN > 90%

for i in unfiltered_data.columns:
    percent=unfiltered_data[i].isna().sum()/len(unfiltered_data)
    if(percent > 0.90):
        unfiltered_data.drop(columns=i,inplace=True)

In [225]:
unfiltered_data.shape

(125306, 127)

## Demographic features

Important! In the paper they mention the following: "As of 30th May, the participants in our data represented 176 different countries. However, there were instances in which we only had one participant per country (i.e. The Bahamas, Uganda, etc.). For computational purposes, we decided to examine the data quality for 42 countries that had over 200 participants."

Therefore, we also need to apply this filter

In [226]:
unfiltered_data.Country.nunique()

177

In [227]:
unfiltered_data = unfiltered_data.groupby('Country').filter(lambda x : len(x)>=200)

In [228]:
unfiltered_data.Country.nunique()

43

In [229]:
unfiltered_data.shape

(122065, 127)

In [230]:
# Take all demographic columns and replace NaN by "missing"

for i in unfiltered_data.loc[:, 'UserLanguage': 'Dem_isolation_kids']:
    unfiltered_data[i].replace(np.nan,'missing',inplace=True)

In [231]:
for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_state 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.07847458321386147
Scale_PSS10_UCLA_2 0.07868758448367673
Scale_PSS10_UCLA_3 0.0783926596485479
Scale_PSS10_UCLA_4 0.07922008765821488
Scale_PSS10_UCLA_5 0.08016220865932086
Scale_PSS10_UCLA_6 0.07838446729201655
Scale_PSS10_UCLA_7 0.07971162905009627
Scale_PSS10_UCLA_8 0.0799655921025683
Scale_PSS10_UCLA_9 0.0780567730307623
Scale_PSS10_UCLA_10 0.07831892843976569
Scale_SLON_1 0.07902347110146234
Scale_SLON_2 0.07958055134559457
Scale_SLON_3 0.07917093351902675
OECD_people_1 0.11075246794740506
OECD_people_2 0.10876172531028551
OECD_insititutions_1 0.12571171097366157
OECD_insititutions_2 0.11872363085241469
OECD_insititutions_3 0.12058329578503256
OECD_insititutions_4 0.11475853029123828
OECD_insititutions_5 0.12

## Stress features (PSS10)

In [232]:
# There are 10 questions assessing the stress levels: 'Scale_PSS10_UCLA_1' to 'Scale_PSS10_UCLA_10';
# Keep rows where participants replied to at least 8 out of 10 questions

unfiltered_data.dropna(how='any', axis=0, thresh=2, subset=['Scale_PSS10_UCLA_1', 'Scale_PSS10_UCLA_2', 'Scale_PSS10_UCLA_3', 'Scale_PSS10_UCLA_4', 'Scale_PSS10_UCLA_5', 'Scale_PSS10_UCLA_6', 'Scale_PSS10_UCLA_7', 'Scale_PSS10_UCLA_8', 'Scale_PSS10_UCLA_9', 'Scale_PSS10_UCLA_10'], inplace=True)

In [233]:
unfiltered_data.shape

(113132, 127)

In [234]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_state 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005922285471838207
Scale_PSS10_UCLA_2 0.006054873952551002
Scale_PSS10_UCLA_3 0.005763179294982852
Scale_PSS10_UCLA_4 0.006664780963829863
Scale_PSS10_UCLA_5 0.007628257257009511
Scale_PSS10_UCLA_6 0.005701304670650214
Scale_PSS10_UCLA_7 0.007150938726443447
Scale_PSS10_UCLA_8 0.007495668776296716
Scale_PSS10_UCLA_9 0.005347735388749425
Scale_PSS10_UCLA_10 0.005648269278365096
Scale_SLON_1 0.00643496093059435
Scale_SLON_2 0.007168617190538486
Scale_SLON_3 0.006762012516352579
OECD_people_1 0.04204822685005127
OECD_people_2 0.03997100731888414
OECD_insititutions_1 0.05819750380086978
OECD_insititutions_2 0.05065763886433547
OECD_insititutions_3 0.052584591450694766
OECD_insititutions_4 0.046370611321288405
OECD_ins

## Expl_Distress 

Set of questions identifying stressors & sources of distress. Assuming that we do not want to drop all the NaN values and we do not want to introduce a bias, these are the possible options: 
- Replace missing values with mode
- Replace missing values with 99.0 (= does not apply to my current situation)

I would go fot the second one, since the mode or most common value could be affected by other variables (country, age, etc). 

In [235]:
# Take all Expl_Distress columns and replace NaN by 99.0 (= does not apply to my current situation)

for i in unfiltered_data.loc[:, 'Expl_Distress_1': 'Expl_Distress_24']:
    unfiltered_data[i].replace(np.nan,99.0,inplace=True)

In [236]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_state 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005922285471838207
Scale_PSS10_UCLA_2 0.006054873952551002
Scale_PSS10_UCLA_3 0.005763179294982852
Scale_PSS10_UCLA_4 0.006664780963829863
Scale_PSS10_UCLA_5 0.007628257257009511
Scale_PSS10_UCLA_6 0.005701304670650214
Scale_PSS10_UCLA_7 0.007150938726443447
Scale_PSS10_UCLA_8 0.007495668776296716
Scale_PSS10_UCLA_9 0.005347735388749425
Scale_PSS10_UCLA_10 0.005648269278365096
Scale_SLON_1 0.00643496093059435
Scale_SLON_2 0.007168617190538486
Scale_SLON_3 0.006762012516352579
OECD_people_1 0.04204822685005127
OECD_people_2 0.03997100731888414
OECD_insititutions_1 0.05819750380086978
OECD_insititutions_2 0.05065763886433547
OECD_insititutions_3 0.052584591450694766
OECD_insititutions_4 0.046370611321288405
OECD_ins

## SPS

Available social provisions in critical/distressing situations. Again, we do not want to drop NaN and we do not want to introduce a bias. In this case there is no "buffer value" as before (does not apply to my current situation). So I would go for the mode.

In [237]:
SPS_mode = unfiltered_data.loc[:, 'SPS_1': 'SPS_10'].apply(lambda x: x.mode(dropna=False)[0])
SPS_mode

SPS_1     6.0
SPS_2     5.0
SPS_3     5.0
SPS_4     5.0
SPS_5     6.0
SPS_6     5.0
SPS_7     6.0
SPS_8     6.0
SPS_9     5.0
SPS_10    6.0
dtype: float64

In [238]:
for i in unfiltered_data.loc[:, 'SPS_1': 'SPS_10']:
    unfiltered_data[i].replace(np.nan,SPS_mode.iloc[0],inplace=True)

In [239]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_state 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005922285471838207
Scale_PSS10_UCLA_2 0.006054873952551002
Scale_PSS10_UCLA_3 0.005763179294982852
Scale_PSS10_UCLA_4 0.006664780963829863
Scale_PSS10_UCLA_5 0.007628257257009511
Scale_PSS10_UCLA_6 0.005701304670650214
Scale_PSS10_UCLA_7 0.007150938726443447
Scale_PSS10_UCLA_8 0.007495668776296716
Scale_PSS10_UCLA_9 0.005347735388749425
Scale_PSS10_UCLA_10 0.005648269278365096
Scale_SLON_1 0.00643496093059435
Scale_SLON_2 0.007168617190538486
Scale_SLON_3 0.006762012516352579
OECD_people_1 0.04204822685005127
OECD_people_2 0.03997100731888414
OECD_insititutions_1 0.05819750380086978
OECD_insititutions_2 0.05065763886433547
OECD_insititutions_3 0.052584591450694766
OECD_insititutions_4 0.046370611321288405
OECD_ins

**Decide what to do with SPS_avg!!!!!!**

## Expl_Coping

16 own self-reported items to capture how people reduce their discomfort on a 6- Point Likert Scale.  Again, we do not want to drop NaN and we do not want to introduce a bias. In this case there is no "buffer value" as before (does not apply to my current situation). So I would go for the mode as before.

In [240]:
Expl_coping_mode = unfiltered_data.loc[:, 'Expl_Coping_1': 'Expl_Coping_16'].apply(lambda x: x.mode(dropna=False)[0])
Expl_coping_mode

Expl_Coping_1     4.0
Expl_Coping_2     5.0
Expl_Coping_3     5.0
Expl_Coping_4     NaN
Expl_Coping_5     5.0
Expl_Coping_6     5.0
Expl_Coping_7     1.0
Expl_Coping_8     1.0
Expl_Coping_9     5.0
Expl_Coping_10    4.0
Expl_Coping_11    4.0
Expl_Coping_12    5.0
Expl_Coping_13    5.0
Expl_Coping_14    5.0
Expl_Coping_15    1.0
Expl_Coping_16    5.0
dtype: float64

**Most common value in Expl_coping_4 is NaN, so it makes no sense to keep this column. The other option is to do inplace=True**

In [241]:
unfiltered_data.drop(columns=['Expl_Coping_4'], inplace=True)

In [242]:
for i in unfiltered_data.loc[:, 'Expl_Coping_1': 'Expl_Coping_16']:
    unfiltered_data[i].replace(np.nan,Expl_coping_mode.iloc[0],inplace=True)

In [243]:
# Print the name of columns & correspondant percentage of NaN

for i in unfiltered_data.columns:
    print(i, unfiltered_data[i].isna().sum()/len(unfiltered_data))

UserLanguage 0.0
Dem_age 0.0
Dem_gender 0.0
Dem_edu 0.0
Dem_edu_mom 0.0
Dem_employment 0.0
Country 0.0
Dem_Expat 0.0
Dem_state 0.0
Dem_maritalstatus 0.0
Dem_dependents 0.0
Dem_riskgroup 0.0
Dem_isolation 0.0
Dem_isolation_adults 0.0
Dem_isolation_kids 0.0
Scale_PSS10_UCLA_1 0.005922285471838207
Scale_PSS10_UCLA_2 0.006054873952551002
Scale_PSS10_UCLA_3 0.005763179294982852
Scale_PSS10_UCLA_4 0.006664780963829863
Scale_PSS10_UCLA_5 0.007628257257009511
Scale_PSS10_UCLA_6 0.005701304670650214
Scale_PSS10_UCLA_7 0.007150938726443447
Scale_PSS10_UCLA_8 0.007495668776296716
Scale_PSS10_UCLA_9 0.005347735388749425
Scale_PSS10_UCLA_10 0.005648269278365096
Scale_SLON_1 0.00643496093059435
Scale_SLON_2 0.007168617190538486
Scale_SLON_3 0.006762012516352579
OECD_people_1 0.04204822685005127
OECD_people_2 0.03997100731888414
OECD_insititutions_1 0.05819750380086978
OECD_insititutions_2 0.05065763886433547
OECD_insititutions_3 0.052584591450694766
OECD_insititutions_4 0.046370611321288405
OECD_ins

## Expl_media

6 own self-reported items to capture which information sources / media channels are used on a 6-Point Likert Scale. In this case questions 1 to 3 are specific to each country, so:
- Questions 1-3: group by country and calculate the mode based on that.
- Questions 4-6: apply mode independently of the country.

In [263]:
Expl_media_456_mode = unfiltered_data.loc[:, 'Expl_media_4': 'Expl_media_6'].apply(lambda x: x.mode(dropna=False)[0])
Expl_media_456_mode

Expl_media_4    4.0
Expl_media_5    5.0
Expl_media_6    NaN
dtype: float64

**What do we do with Q6? Same options as before: drop the column or apply dropna=True and compute the mode with the available values**!

In [258]:
unfiltered_data.groupby('Country').Expl_media_1.apply(lambda x: x.mode(dropna=False)[0])

Country
Argentina                 NaN
Australia                 5.0
Austria                   5.0
Bangladesh                NaN
Belgium                   5.0
Bosnia and Herzegovina    NaN
Brazil                    NaN
Bulgaria                  NaN
Canada                    5.0
Croatia                   5.0
Czech Republic            4.0
Denmark                   5.0
Finland                   5.0
France                    5.0
Germany                   5.0
Greece                    5.0
Hungary                   NaN
Indonesia                 NaN
Ireland                   NaN
Italy                     5.0
Japan                     4.0
Korea, South              5.0
Kosovo                    NaN
Lithuania                 5.0
Malaysia                  5.0
Mexico                    5.0
Netherlands               5.0
Pakistan                  NaN
Panama                    NaN
Philippines               4.0
Poland                    NaN
Portugal                  NaN
Romania                   NaN
Se

In [259]:
unfiltered_data.groupby('Country').Expl_media_2.apply(lambda x: x.mode(dropna=False)[0])

Country
Argentina                 NaN
Australia                 5.0
Austria                   NaN
Bangladesh                NaN
Belgium                   5.0
Bosnia and Herzegovina    NaN
Brazil                    NaN
Bulgaria                  NaN
Canada                    5.0
Croatia                   5.0
Czech Republic            4.0
Denmark                   5.0
Finland                   5.0
France                    5.0
Germany                   5.0
Greece                    5.0
Hungary                   NaN
Indonesia                 NaN
Ireland                   5.0
Italy                     5.0
Japan                     4.0
Korea, South              4.0
Kosovo                    NaN
Lithuania                 5.0
Malaysia                  5.0
Mexico                    5.0
Netherlands               5.0
Pakistan                  NaN
Panama                    NaN
Philippines               4.0
Poland                    NaN
Portugal                  NaN
Romania                   NaN
Se

In [260]:
unfiltered_data.groupby('Country').Expl_media_3.apply(lambda x: x.mode(dropna=False)[0])

Country
Argentina                 NaN
Australia                 5.0
Austria                   NaN
Bangladesh                NaN
Belgium                   5.0
Bosnia and Herzegovina    NaN
Brazil                    NaN
Bulgaria                  NaN
Canada                    5.0
Croatia                   NaN
Czech Republic            NaN
Denmark                   2.0
Finland                   4.0
France                    4.0
Germany                   5.0
Greece                    5.0
Hungary                   NaN
Indonesia                 NaN
Ireland                   NaN
Italy                     NaN
Japan                     4.0
Korea, South              5.0
Kosovo                    NaN
Lithuania                 5.0
Malaysia                  5.0
Mexico                    5.0
Netherlands               2.0
Pakistan                  NaN
Panama                    NaN
Philippines               4.0
Poland                    NaN
Portugal                  NaN
Romania                   NaN
Se

**At this point I would drop the rows in each of the 3 questions where the most common value is NaN based on the country. The other option is to go for dropna=True and simply compute the mode with the available values**

In [261]:
unfiltered_data.groupby('Country').Expl_media_1.apply(lambda x: x.mode(dropna=True)[0])

Country
Argentina                 5.0
Australia                 5.0
Austria                   5.0
Bangladesh                5.0
Belgium                   5.0
Bosnia and Herzegovina    2.0
Brazil                    5.0
Bulgaria                  4.0
Canada                    5.0
Croatia                   5.0
Czech Republic            4.0
Denmark                   5.0
Finland                   5.0
France                    5.0
Germany                   5.0
Greece                    5.0
Hungary                   1.0
Indonesia                 5.0
Ireland                   5.0
Italy                     5.0
Japan                     4.0
Korea, South              5.0
Kosovo                    5.0
Lithuania                 5.0
Malaysia                  5.0
Mexico                    5.0
Netherlands               5.0
Pakistan                  5.0
Panama                    5.0
Philippines               4.0
Poland                    5.0
Portugal                  5.0
Romania                   4.0
Se

## Once this is done we can dropna and see the total values we managed to keep

## Do we need to scale the data?